In [1]:
from transformers import pipeline
from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch
import os

#print(torch.cuda.device_count())
#os.environ["CUDA_VISIBLE_DEVICES"] = "2,3"

#モデル読み込み
model_name = "meta-llama/Llama-2-7b-chat-hf"

model_name = "tokyotech-llm/Swallow-MS-7b-v0.1"
model_name = "cyberagent/open-calm-7b"

model_name="Local-Novel-LLM-project/Vecteus-v1"
#model_name="nvidia/Llama3-ChatQA-1.5-8B"

model_name="../model/------llm-models-hf-2ja1_part_inst_all_lr_1e-5"
model_name="../model/------llm-models-hf-2ja1_part_inst_all_lr_1e-5"
model_name="../X_merge/merged_models/0507test"
#model_name="../X_merge/merged_models/mergoo_llama_test"

model_path="../model/---X_merge-merged_models-0507test_inst_all_lr_1e-5"
tokenizer = AutoTokenizer.from_pretrained(model_name)


/home/hatakeyama/miniconda3/envs/llmeval2/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers
/home/hatakeyama/miniconda3/envs/llmeval2/lib/python3.11/site-packages/transformers/convert_slow_tokenizer.py:560: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [6]:
t_list=[
"\n\n### 応答:\n",
"\n\n### 応答:\n「",
"\n\n### 応答:\nああ",
"\n\n### 応答:\naaa",
"1\n\n### 応答:\naaa",
"あ\n\n### 応答:\naaa",
"」\n\n### 応答:\naaa",
]
t_list=[
"\n\n### 指示:\n",
"\n\n### 指示:\n「",
"\n\n### 指示:\nああ",
"\n\n### 指示:\naaa",
"1\n\n### 指示:\naaa",
"あ\n\n### 指示:\naaa",
"」\n\n### 指示:\naaa",
]
for t in t_list:
    print(tokenizer.encode(t),tokenizer.tokenize(t))


[5092, 272, 3994, 327, 2] ['▁###', '▁', '指示', ':']
[5092, 272, 3994, 327, 337, 2] ['▁###', '▁', '指示', ':', '▁「']
[5092, 272, 3994, 327, 8722, 2] ['▁###', '▁', '指示', ':', '▁ああ']
[5092, 272, 3994, 327, 272, 55841, 2] ['▁###', '▁', '指示', ':', '▁', 'aaa']
[272, 279, 5092, 272, 3994, 327, 272, 55841, 2] ['▁', '1', '▁###', '▁', '指示', ':', '▁', 'aaa']
[4543, 5092, 272, 3994, 327, 272, 55841, 2] ['▁あ', '▁###', '▁', '指示', ':', '▁', 'aaa']
[272, 285, 5092, 272, 3994, 327, 272, 55841, 2] ['▁', '」', '▁###', '▁', '指示', ':', '▁', 'aaa']


In [7]:
tokenizer.encode(t)

[272, 285, 5092, 272, 3994, 327, 272, 55841, 2]

In [ ]:
model = AutoModelForCausalLM.from_pretrained(model_name, 
                                            #quantization_config=bnb_config, 
                                            device_map="auto",
                                            )


In [ ]:
model.load_adapter(model_path)
#from mergoo.models.modeling_llama import LlamaForCausalLM

#model_name="../X_merge/merged_models/mergoo_llama_test"
#model=LlamaForCausalLM.from_pretrained(model_name,device_map="auto")

In [ ]:
model

In [ ]:
#層の表示
for name, param in model.named_parameters():
    print(name)
    if "gate" in name:
        print("***",name)

In [ ]:
def perplexity(model, tokenizer, text) -> torch.Tensor:
    tokenized_input = tokenizer.encode(
        text, add_special_tokens=False, return_tensors="pt"
    ).to(model.device)
    with torch.inference_mode():
        output = model(tokenized_input, labels=tokenized_input)
    ppl = torch.exp(output.loss)
    return ppl.item()



In [ ]:
pipe=pipeline('text-generation',model=model,tokenizer=tokenizer, 
              max_new_tokens=50, 
              repetition_penalty=1.2,
              temperature=0.6,
              )


In [ ]:
questions=[
"こんにちは",
"元気ですか?",
"いい天気ですね",
"明日の天気は?",
"計算してください 1+1はいくつですか｡",
#"フィボナッチ数列を生成するpythonのコードを作成してください",
"ドラえもんの友達はだれですか?",
"レイリー散乱とはなんですか",
"hello!",
"今の天気は晴れです｡この結果をjsonで出力してください｡ ",
"純粋理性批判はたぬきに理解できますか?",
]

for question in questions:
    question_template="以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。<SEP>指示<SEP>"
    inp=f"{question_template}{question}<SEP>応答<SEP>"
    print("--------------------")
    print(question)
    print(pipe(inp)[0]["generated_text"][len(inp):])

In [ ]:
pipe("こんにちは")